## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 784))  # 将图像展平
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3091109 ; accuracy 0.10653333
epoch 1 : loss 2.3086748 ; accuracy 0.106816664
epoch 2 : loss 2.3082392 ; accuracy 0.1073
epoch 3 : loss 2.3078046 ; accuracy 0.10786667
epoch 4 : loss 2.3073714 ; accuracy 0.10836667
epoch 5 : loss 2.3069386 ; accuracy 0.10893334
epoch 6 : loss 2.3065069 ; accuracy 0.10938333
epoch 7 : loss 2.3060756 ; accuracy 0.10988333
epoch 8 : loss 2.3056445 ; accuracy 0.11035
epoch 9 : loss 2.305215 ; accuracy 0.11073333
epoch 10 : loss 2.3047853 ; accuracy 0.111266665
epoch 11 : loss 2.3043556 ; accuracy 0.111666664
epoch 12 : loss 2.303928 ; accuracy 0.112166665
epoch 13 : loss 2.3035011 ; accuracy 0.11258333
epoch 14 : loss 2.3030748 ; accuracy 0.11311667
epoch 15 : loss 2.3026495 ; accuracy 0.113516666
epoch 16 : loss 2.3022246 ; accuracy 0.11393333
epoch 17 : loss 2.301801 ; accuracy 0.11435
epoch 18 : loss 2.3013787 ; accuracy 0.1151
epoch 19 : loss 2.3009562 ; accuracy 0.11558333
epoch 20 : loss 2.300535 ; accuracy 0.11596667
epoch 21 : loss